# Preparing the enviroment and mounting data

In [1]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
#from google.colab import drive
#drive.mount("/content/drive/")
#cd drive/My Drive/SFU/Project4-Spring2021
!ls

bartpy				      known_snps_fullname.txt
cevae_cate_checkpoints_a.npy	      known_snps.txt
cevae_cate_checkpoints_b.npy	      level1data_learnersout.npy
cevae_cate_checkpoints_c.npy	      level1data.npy
cevae_cate_checkpoints_d.npy	      __pycache__
cevae_cate_checkpoints_e.npy	      sample_data
cevae_cate_checkpoints_f.npy	      train.py
cevae_cate_checkpoints.npy	      x_clinical_names.npy
cevae_cate_checkpoints_z.npy	      x_clinical.npy
cevae.py			      x_colnames.npy
CompBioAndSimulated_Datasets_alsoold  x_snps.npy
CompBioAndSimulated_Datasets_old      y.npy
eval.py


In [2]:
!git clone https://github.com/JakeColtman/bartpy.git
!git clone https://github.com/raquelaoki/CompBioAndSimulated_Datasets.git

Cloning into 'bartpy'...
remote: Enumerating objects: 1971, done.
remote: Total 1971 (delta 0), reused 0 (delta 0), pack-reused 1971
Receiving objects: 100% (1971/1971), 11.97 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (1329/1329), done.
Cloning into 'CompBioAndSimulated_Datasets'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 5), reused 11 (delta 4), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [3]:
import pandas as pd
import numpy as np 
from scipy import sparse
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.metrics import roc_curve,roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import sys
sys.path.insert(0,'bartpy/')
sys.path.insert(0,'CompBioAndSimulated_Datasets/')
from bartpy.sklearnmodel import SklearnModel
from fromBEDtoNPY import goPDX

data_preprocessing = False
run_learners = True

# BCCH Data Analysis 

## Data 
There are two options: 
1. Pre-processing: require original files on the server and perform the pre-processing / filtering 
2. Load files saved on local machine 


In [10]:
path = '/content/'
if data_preprocessing:  
  print('Option 1: Starting pre-processing:')
  data = goPDX(final=False)
  #import data_preprossing_functions as dpf
  #y, x_clinical, x_snps, x_colnames, x_clinical_names = dpf.run_preprocessing(path, True)
  #x_clinical = x_clinical.astype('float64')
  #print('Done!')
  #print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
  ##318 (318, 42) (318, 16641) 16641
else: 
  print('Option 2: Reading files')
  data = goPDX()

Option 2: Reading files
Loading final form:
Target Y:  302  items and  187  positive examples
Clinical:  302 items and  41  columns
SNPS:  302 items and  16195  columns


## ParKCa

1. Learners: Deconfounder, BART



In [5]:
import train as parkca
import eval as evaluation
import time 
learners = False
if learners: 
  X1 = preprocessing.MinMaxScaler().fit_transform(data.x_clinical)
  X = np.concatenate([X1,data.x_snps], axis = 1)

  coef = parkca.learners(['DA','noise'],X,data.y,data.x_snps_names, colnamesZ = data.x_clinical_names) #bart is time consuming
  #coef = parkca.learners(['CEVAE'],X,y,x_colnames, colnamesZ = x_clinical_names) #SAVED
  z = np.load("cevae_cate_checkpoints_z.npy")
  a = np.load("cevae_cate_checkpoints_a.npy")
  b = np.load("cevae_cate_checkpoints_b.npy")
  c = np.load("cevae_cate_checkpoints_c.npy")
  d = np.load("cevae_cate_checkpoints_d.npy")
  e = np.load("cevae_cate_checkpoints_e.npy")
  f = np.load("cevae_cate_checkpoints_f.npy")

  print(len(z)+len(a)+len(b)+len(c)+len(d)+len(e)+len(f))
  z = [item for sublist in z for item in sublist]
  a = [item for sublist in a for item in sublist]
  b = [item for sublist in b for item in sublist]
  c = [item for sublist in c for item in sublist]
  d = [item for sublist in d for item in sublist]
  e = [item for sublist in e for item in sublist]
  f = [item for sublist in f for item in sublist]

  cevae_coef = np.concatenate([z,a,b,c,d,e,f], 0)
  print(len(cevae_coef))
  coef['CEVAE'] = cevae_coef
  np.save('level1data',coef)
  level1data = coef
else: 
  level1data = np.load('level1data.npy', allow_pickle=True)
  level1data = pd.DataFrame(level1data, columns = ['causes','DA','noise','cevae'])
  print(level1data.head())

level1data['y'] = 0
level1data['y'] = [1 if i in data.known_snps else 0 for i in level1data['causes'].values]



Instructions for updating:
non-resource variables are not supported in the long term
Available devices  1
Current cuda device  0
        causes         DA     noise      cevae
0  1_rs3820011   0.198838   0.78991 -0.0321922
1  1_rs6605080  -0.271758 -0.735683  -0.232704
2   1_rs383968  0.0702615   2.11437  -0.120306
3  1_rs2743979   -0.14591  0.930926  -0.108644
4  1_rs4648564   0.182521 -0.192133   0.305462


In [24]:
level1data.set_index('causes', inplace=True, drop=True)
level1data = parkca.data_norm(level1data)
roc, output, y_full_prob = parkca.meta_learner(level1data.fillna(0), ['lr','nn','upu','rf'],'y')


Meta-learner: LR
Meta-learner: NN
Meta-learner: UPU
Library Missing
Check: https://github.com/t-sakai-kure/pywsl
Meta-learner: RF


In [27]:
output_potential_causes, output_potential_causes_signal  = [], []
output_missing_causes, output_missing_causes_signal, output_confirmed_causes  = [], [],[]

for i in range(len(output['rf'])): 
    if output['rf'][i]==1: 
        output_potential_causes.append(output.index[i])
        output_potential_causes_signal.append(y_full_prob[i,1])
        if output.index[i] in data.known_snps: 
            output_confirmed_causes.append(output.index[i])           
    elif output['rf'][i]==0 and output.index[i] in data.known_snps: 
        output_missing_causes.append(output.index[i])
        #snps.append(data1.index[i])
        output_missing_causes_signal.append(y_full_prob[i,1])
    else: 
      pass 

In [28]:
print("Missing Causes:", len(output_missing_causes))
print("Confirmed Causes:", len(output_confirmed_causes))
print("New Potential Causes:", len(output_potential_causes)-len(output_confirmed_causes))

np.savetxt('output_missing_causes.txt', output_missing_causes,fmt='%s')
np.savetxt('output_confirmed_causes.txt', output_confirmed_causes,fmt='%s')
np.savetxt('output_potential_causes.txt', output_potential_causes,fmt='%s')
np.savetxt('output_potential_causes_signal.txt', output_potential_causes_signal,fmt='%s')
np.savetxt('output_missing_causes_signal.txt', output_missing_causes_signal,fmt='%s')

Missing Causes: 35
Confirmed Causes: 32
New Potential Causes: 127


# Predictive Models 

In [11]:
output_missing_causes = np.loadtxt('output_missing_causes.txt', dtype=np.str)
output_confirmed_causes = np.loadtxt('output_confirmed_causes.txt', dtype=np.str)
output_potential_causes = np.loadtxt('output_potential_causes.txt', dtype=np.str)
output_potential_causes_signal= np.loadtxt('output_potential_causes_signal.txt', dtype=np.str)
output_missing_causes_signal = np.loadtxt('output_missing_causes_signal.txt', dtype=np.str)

if data_preprocessing:  
  print('Option 1: Starting pre-processing:')
  data = goPDX(final=False)
  #import data_preprossing_functions as dpf
  #y, x_clinical, x_snps, x_colnames, x_clinical_names = dpf.run_preprocessing(path, True)
  #x_clinical = x_clinical.astype('float64')
  #print('Done!')
  #print('Shapes:',len(y), sum(y), x_clinical.shape, x_snps.shape, len(x_colnames), len(x_clinical_names))
  ##318 (318, 42) (318, 16641) 16641
else: 
  print('Option 2: Reading files')
  data = goPDX()

In [65]:
# Option 1: using the 300
# option 2: using the != potential (known)
# option 3: using the confirmedis (-potential - missing)
# option 4: using only clinical information (sex, ..., )
# option 5: Random

from random import sample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

def predict_outcome(y_train, y_test, X_train, X_test, lr):
    w = len(y_train) / y_train.sum()
    sample_weight = np.array([w if i == 1 else 1 for i in y_train])
    if lr: 
      model = LogisticRegression()
      tag = 'Logistic Regression'
    else:
      model = RandomForestClassifier(max_depth=6, random_state=0)
      tag = 'Random Forest'
    model.fit(X_train, y_train, sample_weight=sample_weight)

    y_train_p = model.predict(X_train)
    y_test_p = model.predict(X_test)

    f1_train = f1_score(y_train, y_train_p)
    f1_test = f1_score(y_test, y_test_p)

    p_train = precision_score(y_train, y_train_p)
    p_test = precision_score(y_train, y_train_p)

    r_train = recall_score(y_train, y_train_p)
    r_test = recall_score(y_train, y_train_p)


    return tag, f1_train, p_train, r_train, f1_test, p_test, r_test 

def predictive_models_comp(option, data, filter, typemodel = False):
    
    # print(option)
    if option == 1:
      tag = 'All Potential SNPs + Clinical'
    elif option == 2:      
      tag = 'Known SNPs + Clinical'
    elif option == 3:      
      tag = 'Confirmed SNPs + Clinical'
    elif option == 4:      
      tag = 'Only Clinical'
    else:
      tag = 'Random SNPs + Clinical'

    remove = []
    for i in range(len(data.x_snps_names)):
        if data.x_snps_names[i] not in filter:
            remove.append(i)


    X = np.delete(data.x_snps, remove, axis=1)
    colX = np.delete(data.x_snps_names, remove, axis=0)

    y = data.y
    scaler = MinMaxScaler()
    Z = scaler.fit_transform(data.x_clinical)

    if option == 4:
        X = Z
    else:
        X = np.concatenate([Z, X], axis=1)

    rep = 30
    f1_train, f1_test = np.zeros(rep), np.zeros(rep)
    for seed in range(rep):
      y_train, y_test, X_train, X_test = train_test_split(y, X, test_size=0.3, random_state=seed) #8
      model, f1_train[seed], p_train, r_train, f1_test[seed], p_test, r_test = predict_outcome(y_train, y_test, X_train, X_test,typemodel)
      
    roc = {'type': tag,'model':model, 'f1_train':f1_train.mean(), 'f1_train_sd':np.std(f1_train), 
           'f1_test': f1_test.mean(), 'f1_test_sd':np.std(f1_test)}
    #roc = {'type': tag,'model':model, 'f1_train':f1_train, 'p_train':p_train, 'r_train':r_train,
    #       'f1_test': f1_test, 'p_test':p_test, "r_test":r_test}
    return roc


In [66]:
roc_table = pd.DataFrame(columns=['type','model', 'f1_train', 'f1_train_sd','f1_test','f1_test_sd'])
roc_table = roc_table.append(predictive_models_comp(1, data,output_potential_causes ), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(1, data,output_potential_causes,True ), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(2, data, data.known_snps), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(2, data, data.known_snps,True), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(3, data, output_confirmed_causes), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(3, data, output_confirmed_causes,True), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(4, data, [] ), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(4, data, [] ,True), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(5, data,sample(list(data.x_snps_names),40) ), ignore_index=True)
roc_table = roc_table.append(predictive_models_comp(5, data,sample(list(data.x_snps_names),40),True ), ignore_index=True)

roc_table

,type,model,f1_train,f1_train_sd,f1_test,f1_test_sd
0,All Potential SNPs,Random Forest,0.954704,0.011717,0.786759,0.036269
1,All Potential SNPs,Logistic Regression,0.971956,0.008669,0.711725,0.039134
2,Known SNPs,Random Forest,0.955176,0.012017,0.792458,0.033726
3,Known SNPs,Logistic Regression,0.889759,0.012123,0.786599,0.034322
4,Confirmed SNPs,Random Forest,0.956204,0.009467,0.789531,0.034293
5,Confirmed SNPs,Logistic Regression,0.864247,0.013196,0.784731,0.037986
6,Only Clinical,Random Forest,0.967209,0.008133,0.792456,0.035498
7,Only Clinical,Logistic Regression,0.842284,0.016026,0.797318,0.031750
8,random,Random Forest,0.964118,0.007630,0.787228,0.035105
9,random,Logistic Regression,0.881759,0.015616,0.765805,0.038039


In [62]:
# ADD ONLY SNPS 
# ADD ONLY RANDOM SNPs 
# EXTRACT MOST IMPORTANT CLINICAL FEATURES 
# Model with most important clinical variables - removed correlated variables

list

In [ ]:
TODO: make the correlation plot with all snps and clinical data. 


In [67]:
samples = np.load('snpsback_variants.npy')
np.savetxt('snpsback_variants.txt', samples,'%s')

In [52]:
print(samples[23399], samples[90349])
print(samples[23399+1], samples[90349+1])

3_rs7650615 15_rs150980610
3_rs138342348 15_rs78666809
